In [17]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [18]:
market = Market()
spec = ADatabase("btc_spec_classification")
umod = UniversalModeler()

In [19]:
current_year = 2023

In [20]:
factors = []
factors.append("d1")
factors.append("d2")
factors.append("d3")
factors.append("rolling14")

In [21]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [22]:
included_columns.extend(factors)

In [23]:
training_sets = []

In [24]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
ticker_data = prices
ticker_data.sort_values("date",ascending=True,inplace=True)
ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
ticker_data.dropna(inplace=True)
ticker_data["ticker"] = "BTC"
ticker_data["future"] = ticker_data["adjclose"].shift(-1)
ticker_data["delta"] = (ticker_data["future"] - ticker_data["adjclose"]) / ticker_data["adjclose"]
ticker_data["y"] = [x > 0 for x in ticker_data["delta"]]
ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
ticker_data.dropna(inplace=True)
ticker_data = ticker_data[included_columns]
training_sets.append(ticker_data)

In [25]:
data = pd.concat(training_sets)

In [26]:
data = data.dropna()

In [27]:
data

,year,week,ticker,adjclose,y,d1,d2,d3,rolling14
13,2014,50,BTC,351.792712,False,-0.064348,-15.492858,19.339296,368.388483
14,2014,51,BTC,324.602997,False,-0.077289,0.201103,-1.012980,369.107745
15,2014,52,BTC,324.146572,False,-0.001406,-0.981807,-5.882119,362.485428
16,2015,1,BTC,293.639503,False,-0.094115,65.933264,-68.155001,354.338781
17,2015,2,BTC,281.316140,False,-0.041968,-0.554081,-1.008404,348.616607
...,...,...,...,...,...,...,...,...,...
444,2023,9,BTC,22991.969308,False,-0.038203,-4.432715,4.437405,20651.753520
445,2023,10,BTC,21387.747489,True,-0.069773,0.826376,-1.186427,20976.129497
446,2023,11,BTC,25828.691685,True,0.207640,-3.975925,-5.811280,21601.525822
447,2023,12,BTC,27795.255022,False,0.076139,-0.633313,-0.840713,22358.647872


In [ ]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(current_year,current_year+1)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.recommend_classification_model(training_slice,factors,multioutput=False)
spec.disconnect()

  0%|                                                                                                                                      | 0/1 [00:00<?, ?it/s]

In [30]:
stuff

,year,week,ticker,adjclose,y,d1,d2,d3,rolling14,xgb_prediction,xgb_score,cat_prediction,cat_score,tf_prediction,tf_score,prediction,training_years
0,2023,1,BTC,16866.642020,True,-0.308085,-1.680892,-0.915349,18499.940502,0,0.622642,0,0.622642,0,0.377358,False,4
1,2023,2,BTC,19030.571987,True,0.128296,-1.416432,-0.157333,18446.140259,1,0.622642,1,0.622642,0,0.377358,True,4
2,2023,3,BTC,21754.474330,True,0.143133,0.115643,-1.081644,18630.086010,1,0.622642,1,0.622642,0,0.377358,True,4
3,2023,4,BTC,23086.560268,True,0.061233,-0.572197,-5.947973,18901.308527,1,0.622642,0,0.622642,0,0.377358,False,4
4,2023,5,BTC,23273.128348,False,0.008081,-0.868024,0.517003,19109.112198,1,0.622642,0,0.622642,0,0.377358,False,4
5,2023,6,BTC,22299.014230,True,-0.041856,-6.179370,6.118892,19225.297426,0,0.622642,0,0.622642,0,0.377358,False,4
6,2023,7,BTC,23642.106306,True,0.060231,-2.439014,-0.605297,19660.973556,1,0.622642,0,0.622642,0,0.377358,False,4
7,2023,8,BTC,23905.221819,False,0.011129,-0.815226,-0.665756,20179.092198,1,0.622642,0,0.622642,0,0.377358,False,4
8,2023,9,BTC,22991.969308,False,-0.038203,-4.432715,4.437405,20651.753520,0,0.622642,0,0.622642,0,0.377358,False,4
9,2023,10,BTC,21387.747489,True,-0.069773,0.826376,-1.186427,20976.129497,0,0.622642,0,0.622642,0,0.377358,False,4


In [29]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

KeyError: 'model'

In [ ]:
spec.connect()
models = spec.store("models",stuff)
spec.disconnect()